In [3]:
import gc
import re

import numpy as np
"""
ToyNetwork 재분석
    램프 : X
    집계시간 : 5분
    분석시간 1800~8400
    검지기 : 840개
"""
import pandas as pd

import os

# FIX 값 모음
###################################################################################################################

path = r"C:\VISSIM_Workspace\테스트\램프X\mer 통합"

start_interval = 1800
end_interval = 8400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]

###################################################################################################################

# 함수 모음
###################################################################################################################

# 속도 변화율
def speed_mean(original_df):
    copy_df = original_df.copy()

    # 램프 검지기 제외
    copy_df = copy_df[~copy_df["New_Measurement"].between(900, 910)]

    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도 변화율
def density_mean(speed_df):
    copy_df = speed_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 중차량 혼입률
def heavy_rate(original_df):
    copy_df = original_df.copy()

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="heavy_count")
    )
    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="total_count")
    )
    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup", "New_Measurement"],
        how="left"
    )
    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 동적 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()

    # 실측용량 C(3차로 6600)
    max_capacity = 6600
    entry_saturation_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )
    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    display("entry_saturation_df : ", entry_saturation_df)

    return entry_saturation_df

# 램프 간섭 영향률
def rfr_rate(original_df):
    rfr_df = (
        original_df[original_df["New_Measurement"] == 1] # 60
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="main_line")
    )
    input_df = (
        original_df[original_df["New_Measurement"] == 2] # 59
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="input_ramp")
    )
    output_df = (
        original_df[original_df["New_Measurement"] == 3] # 61
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="output_ramp")
    )
    #main_df["RFR"] = (input_df["input_ramp"] + output_df["output_ramp"])/main_df["main_line"]
    # 유출램프 구현이 안 됐기 때문에 0값으로 fix
    rfr_df = rfr_df.drop(columns=["New_Measurement", "main_line"])
    rfr_df["RFR"] = 0
    return rfr_df

# 진출 원활율
def output_normality(original_df):

    copy_df = original_df.copy()
    #copy_df["F(outrate)"] = 0.88
    copy_df["F(outrate)"] = 0
    return copy_df


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged_df = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "New_Measurement", "rate"]], on=["TimeGroup", "New_Measurement"])
    .merge(entry_saturation_df[["TimeGroup", "New_Measurement", "Phi_진입"]], on=["TimeGroup", "New_Measurement"])
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "F(outrate)"]], on="TimeGroup")
    )

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["F(outrate)"]
    )
    merged_df = modify_frame(merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    modify_df = df.copy()

    modify_df["StartTime"] = modify_df["TimeGroup"].str.split("~").str[0].astype(int)
    modify_df = modify_df[(modify_df["StartTime"] >=1800) &(modify_df["StartTime"] < 5400)]
    modify_df = modify_df[modify_df["New_Measurement"] != 280]
    return modify_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = path
mer_list = sorted(
    [file for file in os.listdir(folder_path) if file.endswith(".mer")],
    key=lambda x: int(re.search(r"(\d+)(?=\.mer$)", x).group()) if re.search(r"(\d+)(?=\.mer$)", x) else 0
)

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


for i in range(len(mer_list)):
    mer_file = mer_list[i]
    print("작업파일 : ", mer_file)
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 속도변화율
                speed_df = speed_mean(original_df)

                # 밀도변화율
                density_df = density_mean(speed_df)

                # 중차량 혼입률
                heavy_df = heavy_rate(original_df)

                # 동적 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 간섭 영향률
                rfr_df = rfr_rate(original_df)

                # 진출 원활율
                normality_df = output_normality(rfr_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                #display("stvm_df : ", stvm_df)
                save_to_excel(stvm_df, folder_path, "STVM", i)


                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                #save_to_excel(z_score_df, folder_path, "환산점수", i)

                # STVM 피봇
                #stvm_pivot_df = pivot_table(stvm_df, "STVM")
                #save_to_excel(stvm_pivot_df, folder_path, "STVM_PIVOT", i)

                # 속도값 피봇
                #speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 메모리 정리
                del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()


작업파일 :  network17_수정_600.mer


'entry_saturation_df : '

,TimeGroup,New_Measurement,entry_volume,Phi_진입
0,1800~2100,1,151,0.274545
1,1800~2100,2,150,0.272727
2,1800~2100,3,146,0.265455
3,1800~2100,4,146,0.265455
4,1800~2100,5,145,0.263636
...,...,...,...,...
6155,8100~8400,276,157,0.285455
6156,8100~8400,277,158,0.287273
6157,8100~8400,278,158,0.287273
6158,8100~8400,279,160,0.290909


STVM_1.xlsx 생성 완료
작업파일 :  network17_수정_1000.mer


'entry_saturation_df : '

...

,TimeGroup,New_Measurement,entry_volume,Phi_진입
0,1800~2100,1,411,0.747273
1,1800~2100,2,414,0.752727
2,1800~2100,3,414,0.752727
3,1800~2100,4,416,0.756364
4,1800~2100,5,416,0.756364
...,...,...,...,...
6155,8100~8400,276,404,0.734545
6156,8100~8400,277,405,0.736364
6157,8100~8400,278,409,0.743636
6158,8100~8400,279,409,0.743636


STVM_4.xlsx 생성 완료
작업파일 :  network17_수정_2200.mer


'entry_saturation_df : '

,TimeGroup,New_Measurement,entry_volume,Phi_진입
0,1800~2100,1,557,1.012727
1,1800~2100,2,552,1.003636
2,1800~2100,3,554,1.007273
3,1800~2100,4,553,1.005455
4,1800~2100,5,557,1.012727
...,...,...,...,...
6155,8100~8400,276,526,0.956364
6156,8100~8400,277,531,0.965455
6157,8100~8400,278,538,0.978182
6158,8100~8400,279,541,0.983636


STVM_5.xlsx 생성 완료
